##  Synthetic Data Generation
THe purpose of this script is to generate synthetic data for our hackathon use case.

* The script generates a CSV file where each column represents a different feature of the data.

* The features are samples from distributions of know data data/modeling_cost_df.csv, in particular the following relationships are modeled per station/Line associations:

In [ ]:
import pandas as pd
from IPython.display import display

import os
import re
import geopandas as gpd  
from shapely.geometry import Polygon  
import numpy as np  
import folium  
import networkx as nx


In [145]:
# load prepared synthetic data 
file_path = './prep_data/modeling_cost_df.csv'
modeling_cost_df = pd.read_csv(file_path)

display(modeling_cost_df)

,Line,Station,min_FlowRate,max_FlowRate,min_PowerConsumption,max_PowerConsumption,min_DRACost,max_DRACost,PolyCoeffPowerConsumption,R2PowerConsumption,PolyCoeffDRACost,R2DRACost
0,14,OW,1360,2730,761.439918,3232.280157,0.0,1486.6704,"[1.8648366203666877, -2107.723544222843]",0.948287,"[1.3522885950456174, -2225.584014701982]",0.868130
1,14,SH,1360,2730,89.669202,2570.215535,0.0,1140.5304,"[1.7704206894810628, -2099.7625977989037]",0.974646,"[1.0576792233110657, -1738.8206636687376]",0.868219
2,14,CB,1360,2730,910.989833,3527.955433,0.0,2352.0216,"[1.982213920270733, -2176.01581683247]",0.937545,"[1.9881276863091668, -2982.5128276156056]",0.949106
3,14,WW,1360,2730,605.865959,3443.736592,0.0,2178.9504,"[2.0781550942365845, -2550.005593044673]",0.950960,"[1.875255469834421, -2828.3823170076157]",0.948936
4,14,MN,1360,2730,685.877837,2538.734240,0.0,1509.7464,"[1.3446024019038005, -1100.1556461253317]",0.998284,"[1.335056866730782, -2039.8239287359274]",0.946361
...,...,...,...,...,...,...,...,...,...,...,...,...
211,78A,GT,1900,4200,1519.113698,8974.265195,NaN,NaN,"[3.199415111938144, -4947.649909225776]",0.988725,0,0.000000
212,78B,OV,1900,4200,0.000000,1135.607772,NaN,NaN,"[-0.08876305055603369, 780.8634937501248]",0.020796,0,0.000000
213,78B,SC,1900,4200,0.000000,138.000699,NaN,NaN,"[-0.005579287475650425, 81.77039906611601]",0.005707,0,0.000000
214,78B,SK,1900,4200,1217.970145,4287.270609,NaN,NaN,"[1.3208096914803233, -636.4396522267806]",0.839715,0,0.000000


In [146]:
# List unique lines
unique_lines = sorted(modeling_cost_df['Line'].unique())
print("Number of unique lines:", len(unique_lines))
print("Unique Lines:", unique_lines, "\n")

# List unique stations
unique_stations = modeling_cost_df['Station'].dropna().unique()
print("Number of unique stations:", len(unique_stations))   
print("Unique Stations:", unique_stations)

Number of unique lines: 13
Unique Lines: ['1', '14', '2A', '2B', '4', '5', '61', '65', '67', '6A', '78A', '78B', '93'] 

Number of unique stations: 100
Unique Stations: ['OW' 'SH' 'CB' 'WW' 'MN' 'AM' 'VP' 'EG' 'PR' 'RI' 'BL' 'DS' 'NV' 'MF'
 'CY' 'DL' 'LK' 'DD' 'WA' 'LD' 'HT' 'PT' 'MK' 'LL' 'CV' 'SL' 'OD' 'CR'
 'YP' 'LB' 'LP' 'DR' 'XP' 'JL' 'NC' 'QU' 'SW' 'ZP' 'WL' 'CC' 'KB' 'EP'
 'BB' 'AP' 'GW' 'BU' 'CM' 'ST' 'MI' 'WC' 'GF' 'VG' 'PL' 'DN' 'ME' 'KG'
 'CK' 'HR' 'CS' 'FW' 'SP' 'LO' 'RT' 'DNS' 'PLN' 'WX' 'TI' 'RQ' 'PV' 'KD'
 'IL' 'NB' 'RA' 'MQ' 'MA' 'SN' 'IN' 'IR' 'GC' 'IO' 'BC' 'LE' 'GO' 'WB'
 'SLS' 'MNN' 'BPS' 'LDN' 'MFS' 'OPS' 'DPS' 'OWN' 'MD' 'FN' 'NL' 'GT' 'OV'
 'SC' 'SK' 'HW']


In [147]:
# Group by Line and Station, then calculate the count of unique values for each column
unique_counts = modeling_cost_df.groupby(['Line', 'Station']).agg({
    'min_PowerConsumption': pd.Series.nunique,
    'max_PowerConsumption': pd.Series.nunique
}).reset_index()

# Rename columns for clarity
unique_counts.rename(columns={
    'min_PowerConsumption': 'Uniquemin_PowerConsumptionCount',
    'max_PowerConsumption': 'UniqueMax_PowerConsumptionCount' 
}, inplace=True)

# Display the result
display(unique_counts)

,Line,Station,Uniquemin_PowerConsumptionCount,UniqueMax_PowerConsumptionCount
0,1,AP,1,1
1,1,BB,1,1
2,1,BU,1,1
3,1,CC,1,1
4,1,CM,1,1
...,...,...,...,...
211,93,TI,1,1
212,93,VG,1,1
213,93,WX,1,1
214,93,YP,1,1


In [148]:
# extract unique combinations of Line and Station from unique_counts, add those from modeling_cost_df where Line = 1
unique_combinations = unique_counts[['Line', 'Station']].drop_duplicates()
# Remove duplicates
unique_combinations = unique_combinations.drop_duplicates() 

In [149]:
# Step1: extract the hierachy of associations between Line and Station (real_world example)
line_station_hierarchy = unique_combinations.groupby('Line')['Station'].unique().reset_index()
line_station_hierarchy.rename(columns={'Station': 'AssociatedStations'}, inplace=True)
line_station_hierarchy

,Line,AssociatedStations
0,1,"[AP, BB, BU, CC, CM, CR, DR, EP, GF, GW, JL, K..."
1,14,"[AM, BL, CB, DS, EG, MN, OW, PR, RI, SH, VP, WW]"
2,2A,"[AP, BU, CC, CK, EP, HR, KB, KG, LB, ME, MI, O..."
3,2B,"[CM, CR, DN, DR, FW, GF, LP, NC, PL, SP, VG, XP]"
4,4,"[AP, BU, CC, CK, CM, CR, CS, DN, DR, EP, FW, G..."
5,5,"[BC, GC, GO, IN, IO, IR, LE, MA, MQ, NB, PR, R..."
6,61,"[AM, BPS, DL, DPS, HT, LDN, MFS, MNN, OPS, OWN..."
7,65,"[CM, DN, LP, PL, XP]"
8,67,"[AP, BU, CC, CK, CM, CR, CS, DNS, DR, EP, FW, ..."
9,6A,"[AM, CB, CV, CY, DD, DL, EG, HT, LD, LK, LL, M..."


In [150]:
G = nx.Graph()
# Add edges to the graph based on the line_station_hierarchy
for _, row in line_station_hierarchy.iterrows():
    stations = list(row['AssociatedStations']) 
    line = row['Line']
    # Add edges in a straight line between all stations in the same line
    for i in range(len(stations)):
        for j in range(i + 1, len(stations)):
            G.add_edge(stations[i], stations[j], line=line)

# find the stations where lines intersect
stations_intersections = {}
for line in line_station_hierarchy['Line']:
    stations = line_station_hierarchy[line_station_hierarchy['Line'] == line]['AssociatedStations'].values[0]
    for station in stations:
        if station not in stations_intersections:
            stations_intersections[station] = []
        stations_intersections[station].append(line)

station_line_hierarchy = pd.DataFrame(stations_intersections.items(), columns=['Station', 'Lines'])

# Display the intersections as a table with 2 columns: Station, Intersecting_Lines
station_line_hierarchy = pd.DataFrame(stations_intersections.items(), columns=['Station', 'AssociatedLines'])
display(station_line_hierarchy)

,Station,AssociatedLines
0,AP,"[1, 2A, 4, 67, 93]"
1,BB,[1]
2,BU,"[1, 2A, 4, 67, 93]"
3,CC,"[1, 2A, 4, 67, 93]"
4,CM,"[1, 2B, 4, 65, 67, 93]"
...,...,...
95,KD,[93]
96,PV,[93]
97,RQ,[93]
98,TI,[93]


In [151]:
# Filter stations with multiple line intersections
stations_with_intersections = [
    {'Station': station, 'n_intersections': len(lines), 'lines_at_intersection': lines}
    for station, lines in stations_intersections.items() if len(lines) > 1
]

# Convert to a DataFrame for tabulation
intersections_df = pd.DataFrame(stations_with_intersections)

# Display the result
display(intersections_df)

,Station,n_intersections,lines_at_intersection
0,AP,5,"[1, 2A, 4, 67, 93]"
1,BU,5,"[1, 2A, 4, 67, 93]"
2,CC,5,"[1, 2A, 4, 67, 93]"
3,CM,6,"[1, 2B, 4, 65, 67, 93]"
4,CR,5,"[1, 2B, 4, 67, 93]"
5,DR,4,"[1, 2B, 4, 67]"
6,EP,4,"[1, 2A, 4, 67]"
7,GF,5,"[1, 2B, 4, 67, 93]"
8,KB,5,"[1, 2A, 4, 67, 93]"
9,LB,5,"[1, 2A, 4, 67, 93]"


In [152]:
# USE EGRID SUBREGIONS FOR STATION COORDINATES
# Load the .kml file containing subregions  
egrid_subreg = gpd.read_file('data\egrid2021-simplified-subregion.kml'  )  
egrid_polygons_df = pd.DataFrame(columns=['Region', 'geometry'])

#iterate though the rows of the DataFrame and print the geometry type of each row
for index, row in egrid_subreg.iterrows(): 
    # Extract the region names from the original DataFrame
    region_name = row.Name

    # Read the sample geometry directly from the geometry column
    region_geometry = gpd.GeoSeries([row.geometry])

    # Extract the individual polygons from the MULTIPOLYGON Z object  
    # Convert the MULTIPOLYGON Z to a list of POLYGON objects
    polygons = [Polygon(p.exterior.coords) for p in region_geometry.iloc[0].geoms]

    # Create a new GeoDataFrame with the extracted polygons
    polygons_gdf = gpd.GeoDataFrame(geometry=polygons, crs=region_geometry.crs)
    # Create a new DataFrame with the region names and their corresponding polygons
    polygons_df = pd.DataFrame({'Region': region_name, 'geometry': polygons})

    # append as row to the egrid_polygons_df 
    egrid_polygons_df = pd.concat([egrid_polygons_df, polygons_df], ignore_index=True)

In [174]:
# remove from egrid_polygons_df Region is any of [PRMS, HIMS, HIOA, AKMS, AKGD]
egrid_polygons_df = egrid_polygons_df[~egrid_polygons_df['Region'].isin(['PRMS', 'HIMS', 'HIOA', 'AKMS', 'AKGD'])]
egrid_polygons_df = egrid_polygons_df.reset_index(drop=True)
egrid_polygons_df

,Region,geometry
0,AZNM,POLYGON Z ((-115.9079389999999 32.633733999142...
1,AZNM,"POLYGON Z ((-108.250332 36.91011299909373 0, -..."
2,CAMX,"POLYGON Z ((-120.054946 34.03736299912447 0, -..."
3,CAMX,"POLYGON Z ((-115.898334 36.00428899910245 0, -..."
4,ERCT,POLYGON Z ((-93.56727099999995 31.178780999163...
...,...,...
61,SRTV,POLYGON Z ((-90.13678699999997 33.721641999128...
62,SRTV,POLYGON Z ((-81.67753499999998 36.588117999096...
63,SRVC,POLYGON Z ((-75.52333549999997 35.775020499104...
64,SRVC,POLYGON Z ((-79.66197499999998 38.567615999080...


In [175]:
# Create a folium map centered around the USA  
map_egrid_subregions = folium.Map(location=[37.0902, -95.7129], zoom_start=4)  

# Add polygons to the map
for _, row in egrid_polygons_df.iterrows():
    folium.GeoJson(
        data=row['geometry'],
        name=row['Region'],
        style_function=lambda x: {'fillColor': 'blue', 'color': 'black', 'weight': 1, 'fillOpacity': 0.5}
    ).add_to(map_egrid_subregions)

# Display the map
map_egrid_subregions

In [ ]:
# Ensure the directory exists
output_dir = './prep_maps'
os.makedirs(output_dir, exist_ok=True)
# Save the map to an HTML file
map_egrid_subregions.save(f'{output_dir}/map_egrid_subregions.html')

In [154]:
from shapely.geometry import LineString, Point
import geopandas as gpd
from scipy.spatial import distance
import random

# Function to sort stations by geospatial proximity
def sort_stations_by_proximity(stations, station_coords):
    sorted_stations = []
    remaining_stations = set(stations)
    
    # Start with the first station
    current_station = stations[0]
    sorted_stations.append(current_station)
    remaining_stations.remove(current_station)
    
    while remaining_stations:
        current_coords = station_coords[current_station]
        # Find the closest station
        next_station = min(remaining_stations, key=lambda s: distance.euclidean(current_coords, station_coords[s]))
        sorted_stations.append(next_station)
        remaining_stations.remove(next_station)
        current_station = next_station
    
    return sorted_stations

#### Start with Line = 1 from lline_station_hierachy. 
* Use the first station or OD geo lon/lat = (-117.88145203779813, 47.00953514323572).
* Use the last station or VG geo lon/lat =-76.7095404454374, 35.0010092136677)
* Continue sampling geo-locations from egrid subregions in North Western USA to South Eastern USA roughly in a startight line and asing them the next station, until you reach the last 2nd station

In [155]:
# Filter Line 1 from line_station_hierarchy
line_1_stations = line_station_hierarchy[line_station_hierarchy['Line'] == '1']['AssociatedStations'].values[0]

# Define starting and ending coordinates
start_coords = (-117.88145203779813, 47.00953514323572)  # JL
end_coords = (-76.7095404454374, 35.0010092136677)       # WL

# Initialize a dictionary to store station coordinates
line_1_station_coords = {}
line_1_station_coords['JL'] = start_coords  # Assign start station
# remove JL and WL from the list of stations to avoid duplication
line_1_stations = line_1_stations[(line_1_stations != 'JL') & (line_1_stations != 'WL')]

# Calculate step size for longitude and latitude with variability
num_intermediate_stations = len(line_1_stations) - 2
lon_step = ((end_coords[0] - start_coords[0]) / (num_intermediate_stations + 1)) 
lat_step = ((end_coords[1] - start_coords[1]) / (num_intermediate_stations + 1))  

# Assign coordinates to intermediate stations
for i, station in enumerate(line_1_stations[1:-1], start=1):
    lon = start_coords[0] + i * lon_step #+ random.uniform(-0.5, 0.5)
    lat = start_coords[1] + i * lat_step #+ random.uniform(-0.5, 0.5)
    line_1_station_coords[station] = (lon, lat)

# Assign end station
line_1_station_coords['WL'] = end_coords

# apply sort_stations_by_proximity(stations, station_coords)
sorted_stations = sort_stations_by_proximity(list(line_1_station_coords.keys()), line_1_station_coords)
line_1_station_coords_sorted = {station: line_1_station_coords[station] for station in sorted_stations}

# Display the sampled coordinates for Line 1 stations
for station, coords in line_1_station_coords_sorted.items():
    print(f"Station Line 1: {station}, Coordinates: {coords}")

Station Line 1: JL, Coordinates: (-117.88145203779813, 47.00953514323572)
Station Line 1: BB, Coordinates: (-116.0913689250868, 46.487425320211024)
Station Line 1: BU, Coordinates: (-114.30128581237545, 45.96531549718633)
Station Line 1: CC, Coordinates: (-112.51120269966412, 45.44320567416163)
Station Line 1: CM, Coordinates: (-110.72111958695278, 44.92109585113694)
Station Line 1: CR, Coordinates: (-108.93103647424145, 44.39898602811224)
Station Line 1: DR, Coordinates: (-107.14095336153011, 43.87687620508754)
Station Line 1: EP, Coordinates: (-105.35087024881878, 43.35476638206285)
Station Line 1: GF, Coordinates: (-103.56078713610744, 42.83265655903815)
Station Line 1: GW, Coordinates: (-101.7707040233961, 42.31054673601345)
Station Line 1: KB, Coordinates: (-99.98062091068476, 41.78843691298876)
Station Line 1: LB, Coordinates: (-98.19053779797343, 41.26632708996406)
Station Line 1: LP, Coordinates: (-96.40045468526209, 40.744217266939366)
Station Line 1: MI, Coordinates: (-94.610

In [156]:
from shapely.geometry import LineString
import geopandas as gpd

# Extract coordinates from line_1_station_coords_sorted
line_1_coords = list(line_1_station_coords_sorted.values())
line_1_geometry = LineString(line_1_coords)
line_1_geo_gdf = gpd.GeoDataFrame([{'Line': '1', 'geometry': line_1_geometry}], crs="EPSG:4326")

# Display the GeoDataFrame
display(line_1_geo_gdf)

,Line,geometry
0,1,"LINESTRING (-117.88145 47.00954, -116.09137 46..."


In [173]:
from shapely.geometry import Point
import random

# Function to find a random point inside a polygon
def get_random_point_in_polygon(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    while True:
        random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(random_point):
            return random_point

In [158]:
all_stations = {**line_1_station_coords_sorted}
all_lines = line_1_geo_gdf.copy()

In [159]:
# Iterate over each line in line_station_hierarchy (starting after row 2)
for _, row in line_station_hierarchy.iloc[1:].iterrows():
    line = row['Line']
    stations = row['AssociatedStations']
    this_line_coords = {}

    # Assign coordinates to stations
    for station in stations:
        if station not in all_stations.keys():
            # Find a polygon that contains the station
            random_row = egrid_polygons_df.sample(n=1).iloc[0]
            polygon = random_row['geometry']
            random_point = get_random_point_in_polygon(polygon)
            if random_point:
                this_line_coords[station] = (random_point.x, random_point.y)
        else:
            # If no polygon contains the station, assign a random coordinate
            this_line_coords[station] = all_stations[station] if station in all_stations else (random.uniform(-125, -66), random.uniform(25, 50))

    # apply sort_stations_by_proximity(stations, station_coords)
    sorted_stations = sort_stations_by_proximity(list(this_line_coords.keys()), this_line_coords)
    this_line_coords_sorted = {station: this_line_coords[station] for station in sorted_stations}

    # Display the sampled coordinates for Line 1 stations
    for station, coords in this_line_coords_sorted.items():
        print(f"Station Line {line}: {station}, Coordinates: {coords}")

    # Extract coordinates from line_1_station_coords_sorted
    line_coords = list(this_line_coords_sorted.values())
    line_geometry = LineString(line_coords)
    line_geo_gdf = gpd.GeoDataFrame([{'Line': line, 'geometry': line_geometry}], crs="EPSG:4326")

    # append
    all_stations = {**all_stations, **this_line_coords_sorted}
    all_lines = pd.concat([all_lines, line_geo_gdf], ignore_index=True)

Station Line 14: AM, Coordinates: (-86.26373742260905, 36.39645095726522)
Station Line 14: SH, Coordinates: (-90.41705300267719, 36.66557635078104)
Station Line 14: WW, Coordinates: (-93.09744262681225, 37.089269425612066)
Station Line 14: EG, Coordinates: (-93.38272273645788, 37.288471596478395)
Station Line 14: RI, Coordinates: (-94.14725477485956, 36.94949617402339)
Station Line 14: VP, Coordinates: (-92.89138153994972, 39.126012130453844)
Station Line 14: OW, Coordinates: (-91.08045418026799, 31.886186633063026)
Station Line 14: MN, Coordinates: (-94.51391396973777, 31.716774363566298)
Station Line 14: PR, Coordinates: (-107.69808002155011, 34.97552030203249)
Station Line 14: DS, Coordinates: (-106.15700816099978, 43.4333092930875)
Station Line 14: BL, Coordinates: (-104.14363266482532, 45.94990901244934)
Station Line 14: CB, Coordinates: (-73.68661454553211, 40.97851180847129)
Station Line 2A: AP, Coordinates: (-98.55366663525716, 29.0597883124366)
Station Line 2A: CK, Coordinates

In [160]:
all_lines_gdf = gpd.GeoDataFrame(all_lines, geometry='geometry', crs="EPSG:4326")
all_lines_gdf

,Line,geometry
0,1,"LINESTRING (-117.88145 47.00954, -116.09137 46..."
1,14,"LINESTRING (-86.26374 36.39645, -90.41705 36.6..."
2,2A,"LINESTRING (-98.55367 29.05979, -91.88391 29.5..."
3,2B,"LINESTRING (-110.72112 44.9211, -108.93104 44...."
4,4,"LINESTRING (-98.55367 29.05979, -91.88391 29.5..."
5,5,"LINESTRING (-97.48228 38.81758, -99.70578 38.2..."
6,61,"LINESTRING (-86.26374 36.39645, -88.16269 37.8..."
7,65,"LINESTRING (-110.72112 44.9211, -110.32504 35...."
8,67,"LINESTRING (-98.55367 29.05979, -91.88391 29.5..."
9,6A,"LINESTRING (-86.26374 36.39645, -90.41705 36.6..."


In [161]:
from shapely.geometry import Point
import geopandas as gpd

# Convert all_stations dictionary to a GeoDataFrame
all_stations_gdf = gpd.GeoDataFrame(
    [{'Station': station, 'geometry': Point(coords)} for station, coords in all_stations.items()],
    crs="EPSG:4326"  # Specify the coordinate reference system
)
all_stations_gdf

,Station,geometry
0,JL,POINT (-117.88145 47.00954)
1,BB,POINT (-116.09137 46.48743)
2,BU,POINT (-114.30129 45.96532)
3,CC,POINT (-112.5112 45.44321)
4,CM,POINT (-110.72112 44.9211)
...,...,...
95,KD,POINT (-89.57893 32.21413)
96,TI,POINT (-90.73866 35.72965)
97,RQ,POINT (-116.24833 33.30604)
98,WX,POINT (-83.34092 41.29167)


### Egrid Subregions

In [162]:
# Ensure egrid_polygons_df is a GeoDataFrame
egrid_polygons_gdf = gpd.GeoDataFrame(egrid_polygons_df, geometry='geometry', crs="EPSG:4326")

# Perform a spatial join to find the eGRID region for each station
stations_with_egrid = gpd.sjoin(all_stations_gdf, egrid_polygons_gdf, how='left', predicate='intersects')

# Rename the 'Region' column to 'egrid_subregion' and drop unnecessary columns
stations_with_egrid.rename(columns={'Region': 'egrid_subregion'}, inplace=True)
stations_with_egrid = stations_with_egrid[['Station', 'geometry', 'egrid_subregion']]

# Display the updated GeoDataFrame
display(stations_with_egrid)

,Station,geometry,egrid_subregion
0,JL,POINT (-117.88145 47.00954),NWPP
1,BB,POINT (-116.09137 46.48743),NWPP
2,BU,POINT (-114.30129 45.96532),NWPP
3,CC,POINT (-112.5112 45.44321),NWPP
4,CM,POINT (-110.72112 44.9211),NWPP
...,...,...,...
96,TI,POINT (-90.73866 35.72965),SPSO
97,RQ,POINT (-116.24833 33.30604),AZNM
98,WX,POINT (-83.34092 41.29167),RFCW
99,PV,POINT (-78.58103 41.61639),RFCE


### Map

In [182]:
# Define a color palette for the lines
color_palette = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'red', 'beige', 'darkblue', 'darkgreen', 'lightgreen', 'cadetblue','lightblue']
line_colors = {line: color_palette[i % len(color_palette)] for i, line in enumerate(all_lines['Line'])}
line_colors

{'1': 'red',
 '14': 'blue',
 '2A': 'green',
 '2B': 'purple',
 '4': 'orange',
 '5': 'darkred',
 '61': 'red',
 '65': 'beige',
 '67': 'darkblue',
 '6A': 'darkgreen',
 '78A': 'lightgreen',
 '78B': 'cadetblue',
 '93': 'lightblue'}

In [183]:
# Visuallize one line on a map  
map_lines = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# select line 1 from all_lines
row = all_lines[all_lines['Line'] == '93']
line = row['Line'].iloc[0]
line_geometry = row['geometry'].iloc[0]
line_color = line_colors.get(line, 'gray')  # Default to gray if no color is assigned

# Add markers for the stations in the current line
for station, coords in all_stations.items():
    if line in stations_intersections.get(station, []):  # Check if the station belongs to the current line
        folium.Marker(
            location=[coords[1], coords[0]],  # Latitude, Longitude
            popup=f"Line {line} Station: {station}",
            tooltip=station,
            icon=folium.Icon(color=line_color, icon='info-sign')
        ).add_to(map_lines)

# Add the line geometry to the map
folium.GeoJson(
    line_geometry,
    name=f'Line {line}',
    style_function=lambda x, color=line_color: {'color': color, 'weight': 5}
).add_to(map_lines)

# Display the map
map_lines

In [184]:
from branca.element import Template, MacroElement

# Visualize ALL lines and stations on a map with selectable legend
map_lines = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add lines and stations to the map
line_layers = {}
station_layers = {}

for _, row in all_lines_gdf.iterrows():
    line = row['Line']
    line_geometry = row['geometry']
    line_color = line_colors.get(line, 'gray')  # Default to gray if no color is assigned

    # Create a feature group for the line
    line_layer = folium.FeatureGroup(name=f"Line {line}", show=True)
    folium.GeoJson(
        line_geometry,
        name=f'Line {line}',
        style_function=lambda x, color=line_color: {'color': color, 'weight': 5}
    ).add_to(line_layer)
    line_layers[line] = line_layer
    map_lines.add_child(line_layer)

    # Create a feature group for the stations of the line
    station_layer = folium.FeatureGroup(name=f"Stations for Line {line}", show=True)
    for _, station_row in stations_with_egrid.iterrows():
        station = station_row['Station']
        coords = station_row['geometry']
        egrid_subregion = station_row['egrid_subregion']
        if line in stations_intersections.get(station, []):  # Check if the station belongs to the current line
            folium.Marker(
                location=[coords.y, coords.x],  # Latitude, Longitude
                popup=f"Line {line} Station: {station} Egrid Subregion: {egrid_subregion}",
                tooltip=station,
                icon=folium.Icon(color=line_color, icon='info-sign')
            ).add_to(station_layer)
    station_layers[line] = station_layer
    map_lines.add_child(station_layer)

# Add layer control
folium.LayerControl(collapsed=False).add_to(map_lines)

# Add a custom legend with JavaScript for toggling lines and stations
legend_html = """
<div id="map-legend" style="position: fixed; 
            bottom: 50px; left: 50px; width: 250px; height: auto; 
            background-color: white; z-index:9999; font-size:14px;
            border:2px solid grey; padding: 10px;">
    <b>Legend</b><br>
    {}
</div>
<script>
function toggleLayer(layerName) {{
    var layers = document.getElementsByClassName('leaflet-control-layers-selector');
    for (var i = 0; i < layers.length; i++) {{
        if (layers[i].nextSibling.innerHTML === layerName) {{
            layers[i].click();
        }}
    }}
}}
</script>
""".format(
    "<br>".join([f"<i style='color:{color}'>&#9679;</i> <a href='#' onclick=\"toggleLayer('Line {line}')\">Line {line}</a>" for line, color in line_colors.items()])
)

legend = MacroElement()
legend._template = Template(legend_html)
map_lines.get_root().add_child(legend)

# Display the map
map_lines

In [185]:
# Ensure the directory exists
output_dir = './prep_maps'
os.makedirs(output_dir, exist_ok=True)
# Save the map to an HTML file
map_lines.save(f'{output_dir}/mocked_network_map.html')

In [166]:
# Save all_lines GeoDataFrame to a file (e.g., GeoJSON format)
output_file_path = './prep_data/all_lines.geojson'
all_lines_gdf.to_file(output_file_path, driver='GeoJSON')

print(f"GeoDataFrame saved to {output_file_path}")

GeoDataFrame saved to ./prep_data/all_lines.geojson


In [167]:
# Save the GeoDataFrame to a file (e.g., GeoJSON format)
output_file_path = './prep_data/all_stations.geojson'
stations_with_egrid.to_file(output_file_path, driver='GeoJSON')

print(f"GeoDataFrame saved to {output_file_path}")

GeoDataFrame saved to ./prep_data/all_stations.geojson


### Create a geospatial network

In [168]:
import networkx as nx
import folium

# Create a graph
G = nx.Graph()

# Add nodes (stations) to the graph
for station, coords in all_stations.items():
    G.add_node(station, pos=coords)

# Add edges (connections between stations) based on all_lines
for _, row in all_lines.iterrows():
    line = row['Line']
    line_geometry = row['geometry']
    if line_geometry and isinstance(line_geometry, LineString):
        coords = list(line_geometry.coords)
        for i in range(len(coords) - 1):
            # Find the stations corresponding to the coordinates
            station1 = next((s for s, c in all_stations.items() if c == coords[i]), None)
            station2 = next((s for s, c in all_stations.items() if c == coords[i + 1]), None)
            if station1 and station2:
                G.add_edge(station1, station2, line=line)

# Create a folium map centered around the USA
map_network = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add nodes (stations) to the map
for station, coords in all_stations.items():
    folium.Marker(
        location=[coords[1], coords[0]],  # Latitude, Longitude
        popup=f"Station: {station}",
        tooltip=station,
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_network)

# Add edges (connections) to the map
for edge in G.edges(data=True):
    station1, station2, data = edge
    coords1 = all_stations[station1]
    coords2 = all_stations[station2]
    folium.PolyLine(
        locations=[[coords1[1], coords1[0]], [coords2[1], coords2[0]]],
        color='red',
        weight=2,
        tooltip=f"Line: {data['line']}"
    ).add_to(map_network)


In [169]:
# Find the largest connected component
largest_cc = max(nx.connected_components(G), key=len)
largest_cc_subgraph = G.subgraph(largest_cc)

# Create a folium map centered around the USA
map_largest_cc = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add nodes (stations) in the largest connected component to the map
for station in largest_cc_subgraph.nodes:
    coords = all_stations[station]
    folium.Marker(
        location=[coords[1], coords[0]],  # Latitude, Longitude
        popup=f"Station: {station}",
        tooltip=station,
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_largest_cc)

# Add edges (connections) in the largest connected component to the map
for edge in largest_cc_subgraph.edges(data=True):
    station1, station2, data = edge
    coords1 = all_stations[station1]
    coords2 = all_stations[station2]
    folium.PolyLine(
        locations=[[coords1[1], coords1[0]], [coords2[1], coords2[0]]],
        color='red',
        weight=2,
        tooltip=f"Line: {data['line']}"
    ).add_to(map_largest_cc)

# Display the map
map_largest_cc